In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import os

# --- Codon Table ---
codon_table = {
    'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L',
    'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L',
    'AUU': 'I', 'AUC': 'I', 'AUA': 'I', 'AUG': 'M',
    'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V',
    'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S',
    'AGU': 'S', 'AGC': 'S', 'CCU': 'P', 'CCC': 'P',
    'CCA': 'P', 'CCG': 'P', 'ACU': 'T', 'ACC': 'T',
    'ACA': 'T', 'ACG': 'T', 'GCU': 'A', 'GCC': 'A',
    'GCA': 'A', 'GCG': 'A', 'UAU': 'Y', 'UAC': 'Y',
    'UAA': '*', 'UAG': '*', 'UGA': '*', 'CAU': 'H',
    'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q', 'AAU': 'N',
    'AAC': 'N', 'AAA': 'K', 'AAG': 'K', 'GAU': 'D',
    'GAC': 'D', 'GAA': 'E', 'GAG': 'E', 'UGU': 'C',
    'UGC': 'C', 'UGG': 'W', 'CGU': 'R', 'CGC': 'R',
    'CGA': 'R', 'CGG': 'R', 'AGA': 'R', 'AGG': 'R',
    'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
}

# --- Sequence Functions ---
def reverse_complement(seq):
    complement = {'A': 'U', 'U': 'A', 'G': 'C', 'C': 'G'}
    return "".join(complement[base] for base in reversed(seq))

def validate_sequence(sequence):
    valid_bases = {'A', 'T', 'G', 'C', 'U'}
    return all(base in valid_bases for base in sequence)

def translate_single_frame(sequence):
    sequence = sequence.replace(" ", "").upper()
    if not validate_sequence(sequence):
        return "Error: Invalid character found. Only 'A', 'T', 'G', 'C', 'U' are allowed."
    if 'T' in sequence:
        sequence = sequence.replace('T', 'U')

    protein = ""
    for i in range(0, len(sequence) - 2, 3):
        codon = sequence[i:i+3]
        amino_acid = codon_table.get(codon, '?')
        if amino_acid == '*':
            break
        protein += amino_acid
    return protein

def translate_all_frames(sequence):
    sequence = sequence.replace(" ", "").upper()
    if not validate_sequence(sequence):
        return {"Error": "Invalid character found. Only 'A', 'T', 'G', 'C', 'U' are allowed."}
    if 'T' in sequence:
        sequence = sequence.replace('T', 'U')

    forward_frames = [sequence[i:] for i in range(3)]
    reverse_seq = reverse_complement(sequence)
    reverse_frames = [reverse_seq[i:] for i in range(3)]

    all_frames = forward_frames + reverse_frames
    frame_labels = ["Forward Frame 1", "Forward Frame 2", "Forward Frame 3",
                    "Reverse Frame 1", "Reverse Frame 2", "Reverse Frame 3"]

    results = {}
    for label, frame in zip(frame_labels, all_frames):
        protein = ""
        start_found = False
        for i in range(0, len(frame) - 2, 3):
            codon = frame[i:i+3]
            if codon == 'AUG':
                start_found = True
                protein = "M"
            elif start_found:
                if codon in {'UAA', 'UAG', 'UGA'}:
                    break
                protein += codon_table.get(codon, '?')
        results[label] = protein if protein else "No valid protein found"
    return results

def read_fasta_or_sequence(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        if not lines:
            raise ValueError("Error: File is empty.")
        if lines[0].startswith('>'):
            return ''.join(line.strip() for line in lines[1:] if not line.startswith('>'))
        return ''.join(line.strip() for line in lines)

# --- GUI Actions ---
def browse_file():
    file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt *.fasta *.fa")])
    if file_path:
        try:
            sequence = read_fasta_or_sequence(file_path)
            sequence_entry.delete("1.0", tk.END)
            sequence_entry.insert(tk.END, sequence)
        except Exception as e:
            messagebox.showerror("File Error", str(e))

def translate_sequence_action():
    sequence = sequence_entry.get("1.0", tk.END).strip()
    if not sequence:
        messagebox.showwarning("Input Missing", "Please enter or load a sequence.")
        return

    output_text.delete("1.0", tk.END)

    if forward_var.get():
        output_text.insert(tk.END, "Forward Translation:\n")
        output_text.insert(tk.END, translate_single_frame(sequence) + "\n\n")

    if reverse_var.get():
        output_text.insert(tk.END, "All Reading Frames:\n")
        frames = translate_all_frames(sequence)
        for frame, result in frames.items():
            output_text.insert(tk.END, f"{frame}: {result}\n")

def save_output():
    content = output_text.get("1.0", tk.END).strip()
    if not content:
        messagebox.showwarning("No Output", "Nothing to save. Please translate first.")
        return
    file_path = filedialog.asksaveasfilename(defaultextension=".txt",
                                             filetypes=[("Text files", "*.txt")])
    if file_path:
        with open(file_path, 'w') as file:
            file.write(content)
        messagebox.showinfo("Saved", f"Output saved to:\n{file_path}")

# --- GUI Setup ---
root = tk.Tk()
root.title("Translator (ORF Tool)")

# Input Frame
input_frame = tk.LabelFrame(root, text="Enter DNA or RNA sequence")
input_frame.pack(padx=10, pady=10, fill="x")

sequence_entry = tk.Text(input_frame, height=5)
sequence_entry.pack(padx=5, pady=5, fill="x")

browse_button = tk.Button(input_frame, text="Browse...", command=browse_file)
browse_button.pack(side="right", padx=5, pady=5)

# Direction Options
direction_frame = tk.Frame(root)
direction_frame.pack(padx=10, pady=5, fill="x")

forward_var = tk.BooleanVar(value=True)
forward_check = tk.Checkbutton(direction_frame, text="Forward", variable=forward_var)
forward_check.pack(side="left")

reverse_var = tk.BooleanVar()
reverse_check = tk.Checkbutton(direction_frame, text="Reverse", variable=reverse_var)
reverse_check.pack(side="left", padx=10)

# Translate Button
translate_button = tk.Button(root, text="Translate", command=translate_sequence_action)
translate_button.pack(pady=10)

# Output Frame
output_frame = tk.LabelFrame(root, text="Output")
output_frame.pack(padx=10, pady=10, fill="both", expand=True)

output_text = tk.Text(output_frame, height=10, width=50)
output_text.pack(padx=5, pady=5, fill="both", expand=True)

# Save Output Button
save_button = tk.Button(root, text="Save Output", command=save_output)
save_button.pack(pady=10)

# Start the GUI
root.mainloop()
